In [13]:
import pandas as pd

# to generate this, click on goodreads "my books".  Then click "export"
books = pd.read_csv("my_liked_books.csv", index_col=0)

In [14]:
books

,user_id,book_id,rating,title
0,-1,127455,0,"The Lies of Locke Lamora (Gentleman Bastard, #1)"
1,-1,17800,0,Dance Dance Dance (The Rat #4)
2,-1,13538873,0,Mr. Penumbra's 24-Hour Bookstore (Mr. Penumbra...
3,-1,641604,0,Purple Cow: Transform Your Business by Being R...
4,-1,122,0,"The Power of One (The Power of One, #1)"
...,...,...,...,...
80,-1,1215032,5,"The Wise Man's Fear (The Kingkiller Chronicle,..."
81,-1,186074,5,The Name of the Wind (The Kingkiller Chronicle...
82,-1,2213661,5,The Graveyard Book
83,-1,15783514,4,The Ocean at the End of the Lane


In [15]:
books["title"] = books["title"].str.replace("title", "")

In [16]:
books

,user_id,book_id,rating,title
0,-1,127455,0,"The Lies of Locke Lamora (Gentleman Bastard, #1)"
1,-1,17800,0,Dance Dance Dance (The Rat #4)
2,-1,13538873,0,Mr. Penumbra's 24-Hour Bookstore (Mr. Penumbra...
3,-1,641604,0,Purple Cow: Transform Your Business by Being R...
4,-1,122,0,"The Power of One (The Power of One, #1)"
...,...,...,...,...
80,-1,1215032,5,"The Wise Man's Fear (The Kingkiller Chronicle,..."
81,-1,186074,5,The Name of the Wind (The Kingkiller Chronicle...
82,-1,2213661,5,The Graveyard Book
83,-1,15783514,4,The Ocean at the End of the Lane


In [17]:
books["rating"] = books["rating"].astype(str)

In [18]:
ratings_string = books["rating"].str.extract("(\[.+\])")

In [19]:
books["rating"] = ratings_string[0].str.slice(start=2, stop=3)

In [20]:
books = books[["title", "rating"]]


In [21]:
titles = pd.read_json("books_titles.json")
titles["book_id"] = titles["book_id"].astype(str)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

tfidf = vectorizer.fit_transform(titles["mod_title"])

In [23]:
import re
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(query,vectorizer):
    processed = re.sub("[^a-zA-Z0-9 ]", "", query.lower())
    query_vec = vectorizer.transform([query])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -10)[-10:]
    results = titles.iloc[indices]
    results = results.sort_values("ratings", ascending=False)
    
    return results.iloc[0,:]["book_id"]

In [27]:
books["book_id"] = books["title"].apply(lambda x: search(x, vectorizer))

/var/folders/vh/4z01kv1n265gqdswjfjqqk2w0000gn/T/ipykernel_7519/2191805772.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books["book_id"] = books["Title"].apply(lambda x: search(x, vectorizer))


In [28]:
books["user_id"] = "-1"


/var/folders/vh/4z01kv1n265gqdswjfjqqk2w0000gn/T/ipykernel_7519/295120743.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books["user_id"] = "-1"


In [33]:
books = books[["user_id", "book_id", "rating", "title"]]


In [36]:
books = books[~pd.isnull(books["rating"])]

In [37]:
books.head()

,user_id,book_id,My Rating,Title


In [ ]:
books.to_csv("my_books.csv")